In [144]:
import json
import os
import pandas as pd
import sys
module_path = os.path.abspath(os.path.join('/Users/juanmanuelpaiba/Documents/Juan_Paiba/new_or_used_algorithm_MELI/', 'python_scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utilities_meli # type: ignore
from feature_engine.encoding import RareLabelEncoder

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

os.chdir(path="/Users/juanmanuelpaiba/Documents/Juan_Paiba/new_or_used_algorithm_MELI")

# You can safely assume that `build_dataset` is correctly implemented
def build_dataset():
    data = [json.loads(x) for x in open("data/Inputs/MLA_100k_checked_v3.jsonlines")]
    target = lambda x: x.get("condition")
    N = -80000
    X_train = data[:N]
    X_test = data[N:]
    y_train = [target(x) for x in X_train]
    y_test = [target(x) for x in X_test]
    for x in X_test:
        del x["condition"]
    return X_train, y_train, X_test, y_test


if __name__ == "__main__":
    print("Loading dataset...")
    # Train and test data following sklearn naming conventions
    # X_train (X_test too) is a list of dicts with information about each item.
    # y_train (y_test too) contains the labels to be predicted (new or used).
    # The label of X_train[i] is y_train[i].
    # The label of X_test[i] is y_test[i].
    X_train, y_train, X_test, y_test = build_dataset()

    # Insert your code below this line:
    # ...

Loading dataset...


In [145]:
# Convert X_test and y_test into a DataFrame

df_test = pd.DataFrame(X_test)
# Reemplazar 'new' por 1 y 'used' por 0 en la df_ajust
#df_test['target'] = df_test['condition'].replace({'new': 1, 'used': 0})
df_test['condition'] = y_test
df_productos = df_test.copy()

In [146]:
df_test.head()

,seller_address,warranty,sub_status,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,variations,site_id,listing_type_id,price,attributes,buying_mode,tags,listing_source,parent_item_id,coverage_areas,category_id,descriptions,last_updated,international_delivery_mode,pictures,id,official_store_id,differential_pricing,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,sold_quantity,available_quantity,condition
0,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],4500.00,"{'local_pick_up': False, 'methods': [], 'tags'...","[{'description': 'Transferencia bancaria', 'id...",7459755121,[],MLA,free,4500.00,[],buy_it_now,[],,None,[],MLA5230,[{'id': 'MLA5631319472-902055524'}],2015-08-23T23:29:58.000Z,none,"[{'size': '281x500', 'secure_url': 'https://a2...",MLA5631319472,NaN,None,True,NaN,ARS,http://mla-s2-p.mlstatic.com/907601-MLA5631319...,Ingletadora Makita,False,2015-08-23T23:28:48.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1445556527000,active,None,NaN,None,1,1440372527000,http://articulo.mercadolibre.com.ar/MLA5631319...,0,1,used
1,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],250.00,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Efectivo', 'id': 'MLAMO', 't...",3353894927,[{'attribute_combinations': [{'value_id': '920...,MLA,bronze,250.00,"[{'value_id': 'male', 'attribute_group_id': 'D...",buy_it_now,[dragged_bids_and_visits],,MLA1717692298,[],MLA64657,[{'id': 'MLA9145362488-926993712'}],2015-10-11T00:20:44.000Z,none,"[{'size': '375x500', 'secure_url': 'https://a2...",MLA9145362488,NaN,None,True,NaN,ARS,http://mla-s2-p.mlstatic.com/668201-MLA9145362...,Camisa Abercrombie Hombre,False,2015-09-22T14:10:15.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1448115015000,active,None,NaN,None,1,1442931015000,http://articulo.mercadolibre.com.ar/MLA9145362...,0,1,used
2,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],1366.00,"{'local_pick_up': False, 'free_methods': [{'ru...",[],7491637146,[],MLA,gold_special,1366.00,"[{'value_id': '', 'attribute_group_id': 'DFLT'...",buy_it_now,[dragged_bids_and_visits],,MLA6597703810,[],MLA60611,[{'id': 'MLA5380847623-925723592'}],2015-10-03T15:46:18.000Z,none,"[{'size': '500x500', 'secure_url': 'https://a2...",MLA5380847623,NaN,None,True,NaN,ARS,http://mla-s2-p.mlstatic.com/897301-MLA5380847...,Kit X2 Amortiguador Trasero Fric Rot Toyota Hi...,True,2015-09-21T03:19:47.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1447989587000,active,QQNfOicE_o8,NaN,None,10,1442805587000,http://articulo.mercadolibre.com.ar/MLA5380847...,0,10,new
3,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],119.99,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",6855821123,[{'attribute_combinations': [{'value_id': '920...,MLA,silver,119.99,"[{'value_id': 'female', 'attribute_group_id': ...",buy_it_now,[dragged_bids_and_visits],,MLA6650761690,[],MLA7660,[{'id': 'MLA6167027559-927291968'}],2015-09-22T19:07:47.000Z,none,"[{'size': '478x457', 'secure_url': 'https://a2...",MLA6167027559,NaN,None,True,NaN,ARS,http://mla-s1-p.mlstatic.com/9412-MLA616702755...,Hermosas Carteritas En Color Detalle Dorado,False,2015-09-22T19:07:44.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1448132864000,active,None,NaN,None,2,1442948864000,http://articulo.mercadolibre.com.ar/MLA6167027...,3,2,new
4,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],100.00,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Efectivo', 'id': 'MLAMO', 't...",8724762628,[],MLA,bronze,100.00,[],buy_it_now,[dragged_bids_and_visits],,MLA9513949068,[],MLA2470,[{'id': 'MLA7416095811-913380895'}],2015-09-06T19:24:28.000Z,none,"[{'size': '498x385', 'secure_url': 'https://a2...",MLA7416095811,NaN,None,True,NaN,A

In [147]:
df_seller = pd.json_normalize(df_productos['seller_address'])
df_seller.rename(columns={'country.name': 'country_name', 'country.id': 'country_id', 'state.name': 
                          'state_name', 'state.id': 'state_id', 'city.name': 'city_name', 'city.id': 'city_id' }, inplace=True)

#Shipping
df_shipping = pd.json_normalize(df_productos['shipping'])
df_shipping = df_shipping.drop(columns=['tag'], errors='ignore')

#attributes
df_attributes = pd.json_normalize(df_productos['attributes'])
df_attr_0 = pd.json_normalize(df_attributes[0])
df_attr_1 = pd.json_normalize(df_attributes[1])
df_attr_0.rename(columns={'value_name': 'season_name'}, inplace=True)
df_attr_0 = df_attr_0.drop(columns=['value_id', 'attribute_group_id', 'name', 'attribute_group_name','id'], errors='ignore')
df_attr_1.rename(columns={'value_name': 'gender_name'}, inplace=True)
df_attr_1 = df_attr_1.drop(columns=['value_id', 'attribute_group_id', 'name', 'attribute_group_name','id'], errors='ignore')

#Mdo Pago
df_mdopgo = pd.json_normalize(df_productos['non_mercado_pago_payment_methods'])
#Ajusto 0 las siguientes hasta 11 elimino por valores > Missing 
df_0 = pd.json_normalize(df_mdopgo[0])
df_0.rename(columns={'description': 'descrip_mdo_0', 'id': 'id_mdo_0', 'type': 'type_mdo_0'}, inplace=True)

#Elimino variables iniciales ajustadas
var_drop = ['seller_address', 'shipping', 'non_mercado_pago_payment_methods', 'pictures','attributes']
df_tabular = df_productos.drop(var_drop, axis=1)

# Concat df_ajust
df_productos_00 = pd.concat([df_tabular, df_seller, df_shipping, df_0, df_attr_0,df_attr_1], axis=1)

# Reemplazar 'new' por 1 y 'used' por 0 en la df_ajust
#df_productos_00['target'] = df_productos_00['condition'].replace({'new': 1, 'used': 0})

df_productos_00 = df_productos_00.drop(columns=['sub_status', 'deal_ids', 'variations', 'tags', 'listing_source',
                   'coverage_areas', 'descriptions', 'thumbnail', 'secure_thumbnail', 'methods', 'free_methods'], errors='ignore')

In [148]:
df_productos_00['warranty_cleaned'] = df_productos_00['warranty'].apply(lambda x: utilities_meli.clean_warranty(x))
df_productos_00['warranty_type'] = df_productos_00['warranty'].apply(lambda x: utilities_meli.classify_condition(x))

In [149]:
df_productos_00['first_two_words_title'] = df_productos_00['title'].apply(utilities_meli.extract_first_two_words)
df_productos_00['first_word_title'] = df_productos_00['title'].apply(utilities_meli.extract_first_word)
df_productos_00['first_three_words_title'] = df_productos_00['title'].apply(utilities_meli.extract_first_three_words)
df_productos_00['title_type'] = df_productos_00['title'].apply(lambda x: utilities_meli.classify_condition(x))

In [150]:
rare_values_to_replace = [-2147483648, 11111111, 1111111111, 8888888, 9000000, 123456789, 112111111]

# Replace rare values in 'base_price' column
df_products_00 = utilities_meli.replace_rare_values(df_productos_00, 'base_price', rare_values_to_replace)
df_products_00 = utilities_meli.replace_rare_values(df_products_00, 'base_price', rare_values_to_replace)

# 'date_created' a tipo datetime
df_products_00['date_created_month'] = pd.to_datetime(df_products_00['date_created']).dt.strftime('%Y%m')
df_products_00['date_created'] = pd.to_datetime(df_products_00['date_created'])
# Variables Día - Mes
df_products_00['month'] = df_products_00['date_created'].dt.month
df_products_00['weekday'] = df_products_00['date_created'].dt.weekday
# Variable 'year_month' tipo texto
df_products_00['year_month'] = df_products_00['date_created'].dt.strftime('%Y-%m')
# status
df_products_00['concat_status'] = df_products_00['year_month'] + '_' + df_products_00['status']
# listing_type_id
df_products_00['concat_var_lt'] = df_products_00['year_month'] + '_' + df_products_00['listing_type_id']
# state_id
df_products_00['concat_var_state'] = df_products_00['year_month'] + '_' + df_products_00['state_id']
# automatic_relist
df_products_00['automatic_relist_str'] = df_products_00['automatic_relist'].astype(str)
df_products_00['concat_var_autrelist'] = df_products_00['year_month'] + '_' + df_products_00['automatic_relist_str']
# accepts_mercadopago
df_products_00['accepts_mercadopago_str'] = df_products_00['accepts_mercadopago'].astype(str)
df_products_00['concat_var_accmdopag'] = df_products_00['year_month'] + '_' + df_products_00['accepts_mercadopago_str']
# local_pick_up
df_products_00['local_pick_up_str'] = df_products_00['local_pick_up'].astype(str)
df_products_00['concat_var_localpu'] = df_products_00['year_month'] + '_' + df_products_00['local_pick_up_str']
# free_shipping
df_products_00['free_shipping_str'] = df_products_00['free_shipping'].astype(str)
df_products_00['concat_var_freesh'] = df_products_00['year_month'] + '_' + df_products_00['free_shipping_str']

df_products_00= df_products_00.drop(columns=['permalink','seller_id','warranty','condition','site_id','international_delivery_mode',
                                      'parent_item_id','last_updated','id','title','catalog_product_id',
                                      'dimensions','city_name','stop_time', 'start_time'])

In [151]:
df_products_00 = df_products_00.drop(columns=['differential_pricing','original_price',
                                              'official_store_id','date_created'])

In [152]:
columns_to_group = ['mode', 'status', 'listing_type_id', 'state_id', 'automatic_relist',
                    'accepts_mercadopago', 'local_pick_up', 'free_shipping', 'warranty_cleaned', 'weekday','title_type','warranty_type']

value_column = 'base_price'
value_column_1 = 'initial_quantity'
value_column_2 = 'sold_quantity'

for column in columns_to_group:
    df_products_00 = utilities_meli.calculate_group_stats(df_products_00, column, value_column)
    df_products_00 = utilities_meli.calculate_group_stats(df_products_00, column, value_column_1)
    df_products_00 = utilities_meli.calculate_group_stats(df_products_00, column, value_column_2)

In [153]:
df_products_00.head()

,base_price,listing_type_id,price,buying_mode,category_id,accepts_mercadopago,currency_id,automatic_relist,status,video_id,subtitle,initial_quantity,sold_quantity,available_quantity,country_name,country_id,state_name,state_id,city_id,local_pick_up,free_shipping,mode,descrip_mdo_0,id_mdo_0,type_mdo_0,season_name,gender_name,warranty_cleaned,warranty_type,first_two_words_title,first_word_title,first_three_words_title,title_type,date_created_month,month,weekday,year_month,concat_status,concat_var_lt,concat_var_state,automatic_relist_str,concat_var_autrelist,accepts_mercadopago_str,concat_var_accmdopag,local_pick_up_str,concat_var_localpu,free_shipping_str,concat_var_freesh,mean_base_price_mode,min_base_price_mode,max_base_price_mode,var_base_price_mode,median_base_price_mode,std_base_price_mode,q1_base_price_mode,q3_base_price_mode,mean_initial_quantity_mode,min_initial_quantity_mode,max_initial_quantity_mode,var_initial_quantity_mode,median_initial_quantity_mode,std_initial_quantity_mode,q1_initial_quantity_mode,q3_initial_quantity_mode,mean_sold_quantity_mode,min_sold_quantity_mode,max_sold_quantity_mode,var_sold_quantity_mode,median_sold_quantity_mode,std_sold_quantity_mode,q1_sold_quantity_mode,q3_sold_quantity_mode,mean_base_price_status,min_base_price_status,max_base_price_status,var_base_price_status,median_base_price_status,std_base_price_status,q1_base_price_status,q3_base_price_status,mean_initial_quantity_status,min_initial_quantity_status,max_initial_quantity_status,var_initial_quantity_status,median_initial_quantity_status,std_initial_quantity_status,q1_initial_quantity_status,q3_initial_quantity_status,mean_sold_quantity_status,min_sold_quantity_status,max_sold_quantity_status,var_sold_quantity_status,median_sold_quantity_status,std_sold_quantity_status,q1_sold_quantity_status,q3_sold_quantity_status,mean_base_price_listing_type_id,min_base_price_listing_type_id,max_base_price_listing_type_id,var_base_price_listing_type_id,median_base_price_listing_type_id,std_base_price_listing_type_id,q1_base_price_listing_type_id,q3_base_price_listing_type_id,mean_initial_quantity_listing_type_id,min_initial_quantity_listing_type_id,max_initial_quantity_listing_type_id,var_initial_quantity_listing_type_id,median_initial_quantity_listing_type_id,std_initial_quantity_listing_type_id,q1_initial_quantity_listing_type_id,q3_initial_quantity_listing_type_id,mean_sold_quantity_listing_type_id,min_sold_quantity_listing_type_id,max_sold_quantity_listing_type_id,var_sold_quantity_listing_type_id,median_sold_quantity_listing_type_id,std_sold_quantity_listing_type_id,q1_sold_quantity_listing_type_id,q3_sold_quantity_listing_type_id,mean_base_price_state_id,min_base_price_state_id,max_base_price_state_id,var_base_price_state_id,median_base_price_state_id,std_base_price_state_id,q1_base_price_state_id,q3_base_price_state_id,mean_initial_quantity_state_id,min_initial_quantity_state_id,max_initial_quantity_state_id,var_initial_quantity_state_id,median_initial_quantity_state_id,std_initial_quantity_state_id,q1_initial_quantity_state_id,q3_initial_quantity_state_id,mean_sold_quantity_state_id,min_sold_quantity_state_id,max_sold_quantity_state_id,var_sold_quantity_state_id,median_sold_quantity_state_id,std_sold_quantity_state_id,q1_sold_quantity_state_id,q3_sold_quantity_state_id,mean_base_price_automatic_relist,min_base_price_automatic_relist,max_base_price_automatic_relist,var_base_price_automatic_relist,median_base_price_automatic_relist,std_base_price_automatic_relist,q1_base_price_automatic_relist,q3_base_price_automatic_relist,mean_initial_quantity_automatic_relist,min_initial_quantity_automatic_relist,max_initial_quantity_automatic_relist,var_initial_quantity_automatic_relist,median_initial_quantity_automatic_relist,std_initial_quantity_automatic_relist,q1_initial_quantity_automatic_relist,q3_initial_quantity_automatic_relist,mean_sold_quantity_automatic_relist,min_sold_quantity_automatic_relist,max_sold_quantity_automatic_relist,var_sold_qu

In [154]:
# variables categóricas a transformar
var_dummies =['listing_type_id', 'buying_mode', 'category_id', 'currency_id',
            'status', 'video_id', 'country_name', 'country_id', 'state_name',
            'state_id', 'city_id', 'mode', 'descrip_mdo_0', 'id_mdo_0',
            'type_mdo_0', 'season_name', 'gender_name', 'warranty_cleaned',
            'first_two_words_title', 'first_word_title', 'first_three_words_title',
            'title_type', 'date_created_month', 'year_month', 'concat_status',
            'concat_var_lt', 'concat_var_state', 'automatic_relist_str',
            'concat_var_autrelist', 'accepts_mercadopago_str',
            'concat_var_accmdopag', 'local_pick_up_str', 'concat_var_localpu',
            'free_shipping_str', 'concat_var_freesh']

# groups rare or infrequent categories in a new category called “Rare”, or any other name entered by the user
encoder = RareLabelEncoder(tol=0.03, n_categories=2, variables=var_dummies, replace_with='Rare', missing_values='ignore')

encoder.fit(df_products_00)
# transform the data
df_products_01 = encoder.transform(df_products_00)

In [155]:
df_products_01.head()

,base_price,listing_type_id,price,buying_mode,category_id,accepts_mercadopago,currency_id,automatic_relist,status,video_id,subtitle,initial_quantity,sold_quantity,available_quantity,country_name,country_id,state_name,state_id,city_id,local_pick_up,free_shipping,mode,descrip_mdo_0,id_mdo_0,type_mdo_0,season_name,gender_name,warranty_cleaned,warranty_type,first_two_words_title,first_word_title,first_three_words_title,title_type,date_created_month,month,weekday,year_month,concat_status,concat_var_lt,concat_var_state,automatic_relist_str,concat_var_autrelist,accepts_mercadopago_str,concat_var_accmdopag,local_pick_up_str,concat_var_localpu,free_shipping_str,concat_var_freesh,mean_base_price_mode,min_base_price_mode,max_base_price_mode,var_base_price_mode,median_base_price_mode,std_base_price_mode,q1_base_price_mode,q3_base_price_mode,mean_initial_quantity_mode,min_initial_quantity_mode,max_initial_quantity_mode,var_initial_quantity_mode,median_initial_quantity_mode,std_initial_quantity_mode,q1_initial_quantity_mode,q3_initial_quantity_mode,mean_sold_quantity_mode,min_sold_quantity_mode,max_sold_quantity_mode,var_sold_quantity_mode,median_sold_quantity_mode,std_sold_quantity_mode,q1_sold_quantity_mode,q3_sold_quantity_mode,mean_base_price_status,min_base_price_status,max_base_price_status,var_base_price_status,median_base_price_status,std_base_price_status,q1_base_price_status,q3_base_price_status,mean_initial_quantity_status,min_initial_quantity_status,max_initial_quantity_status,var_initial_quantity_status,median_initial_quantity_status,std_initial_quantity_status,q1_initial_quantity_status,q3_initial_quantity_status,mean_sold_quantity_status,min_sold_quantity_status,max_sold_quantity_status,var_sold_quantity_status,median_sold_quantity_status,std_sold_quantity_status,q1_sold_quantity_status,q3_sold_quantity_status,mean_base_price_listing_type_id,min_base_price_listing_type_id,max_base_price_listing_type_id,var_base_price_listing_type_id,median_base_price_listing_type_id,std_base_price_listing_type_id,q1_base_price_listing_type_id,q3_base_price_listing_type_id,mean_initial_quantity_listing_type_id,min_initial_quantity_listing_type_id,max_initial_quantity_listing_type_id,var_initial_quantity_listing_type_id,median_initial_quantity_listing_type_id,std_initial_quantity_listing_type_id,q1_initial_quantity_listing_type_id,q3_initial_quantity_listing_type_id,mean_sold_quantity_listing_type_id,min_sold_quantity_listing_type_id,max_sold_quantity_listing_type_id,var_sold_quantity_listing_type_id,median_sold_quantity_listing_type_id,std_sold_quantity_listing_type_id,q1_sold_quantity_listing_type_id,q3_sold_quantity_listing_type_id,mean_base_price_state_id,min_base_price_state_id,max_base_price_state_id,var_base_price_state_id,median_base_price_state_id,std_base_price_state_id,q1_base_price_state_id,q3_base_price_state_id,mean_initial_quantity_state_id,min_initial_quantity_state_id,max_initial_quantity_state_id,var_initial_quantity_state_id,median_initial_quantity_state_id,std_initial_quantity_state_id,q1_initial_quantity_state_id,q3_initial_quantity_state_id,mean_sold_quantity_state_id,min_sold_quantity_state_id,max_sold_quantity_state_id,var_sold_quantity_state_id,median_sold_quantity_state_id,std_sold_quantity_state_id,q1_sold_quantity_state_id,q3_sold_quantity_state_id,mean_base_price_automatic_relist,min_base_price_automatic_relist,max_base_price_automatic_relist,var_base_price_automatic_relist,median_base_price_automatic_relist,std_base_price_automatic_relist,q1_base_price_automatic_relist,q3_base_price_automatic_relist,mean_initial_quantity_automatic_relist,min_initial_quantity_automatic_relist,max_initial_quantity_automatic_relist,var_initial_quantity_automatic_relist,median_initial_quantity_automatic_relist,std_initial_quantity_automatic_relist,q1_initial_quantity_automatic_relist,q3_initial_quantity_automatic_relist,mean_sold_quantity_automatic_relist,min_sold_quantity_automatic_relist,max_sold_quantity_automatic_relist,var_sold_qu

In [156]:
###########################################
## Paso Variables Categoricas a Dummies
###########################################
df_products_01 = pd.get_dummies(df_products_01)
# Reemplazar NaN con 0
df_products_01 = df_products_01.fillna(0)
# Convertir True y False a 1 y 0
df_products_01 = df_products_01.astype(int)
df_products_01['subtitle'] = 0

In [157]:
df_products_01 = utilities_meli.create_transformed_columns(df_products_01,'price')
df_products_01 = utilities_meli.create_transformed_columns(df_products_01,'initial_quantity')
df_products_01 = utilities_meli.create_transformed_columns(df_products_01,'sold_quantity')
df_products_01 = utilities_meli.create_transformed_columns(df_products_01,'available_quantity')
df_products_01 = utilities_meli.create_transformed_columns(df_products_01,'var_base_price_mode')
df_products_01 = utilities_meli.create_transformed_columns(df_products_01,'var_base_price_title_type')
df_products_01 = utilities_meli.create_transformed_columns(df_products_01,'var_base_price_warranty_cleaned')

In [158]:
df_products_01['price_x_initial_quantity'] = df_products_01['price']*df_products_01['initial_quantity']
df_products_01['price_x_sold_quantity'] = df_products_01['price']*df_products_01['sold_quantity']
df_products_01['price_2_x_sold_quantity'] = df_products_01['price_square']*df_products_01['sold_quantity']
df_products_01['price_x_initial_quantity'] = df_products_01['price']*df_products_01['initial_quantity']
df_products_01['price_x_sold_quantity'] = df_products_01['price']*df_products_01['sold_quantity']
df_products_01['price_4_x_sold_quantity'] = df_products_01['price_fourth']*df_products_01['sold_quantity']
df_products_01['price_4_x_initial_quantity'] = df_products_01['price_fourth']*df_products_01['initial_quantity']

In [159]:
print(df_products_01.shape)

(80000, 511)


In [160]:
df_products_01.head()

,base_price,price,accepts_mercadopago,automatic_relist,initial_quantity,sold_quantity,available_quantity,local_pick_up,free_shipping,month,weekday,mean_base_price_mode,min_base_price_mode,max_base_price_mode,var_base_price_mode,median_base_price_mode,std_base_price_mode,q1_base_price_mode,q3_base_price_mode,mean_initial_quantity_mode,min_initial_quantity_mode,max_initial_quantity_mode,var_initial_quantity_mode,median_initial_quantity_mode,std_initial_quantity_mode,q1_initial_quantity_mode,q3_initial_quantity_mode,mean_sold_quantity_mode,min_sold_quantity_mode,max_sold_quantity_mode,var_sold_quantity_mode,median_sold_quantity_mode,std_sold_quantity_mode,q1_sold_quantity_mode,q3_sold_quantity_mode,mean_base_price_status,min_base_price_status,max_base_price_status,var_base_price_status,median_base_price_status,std_base_price_status,q1_base_price_status,q3_base_price_status,mean_initial_quantity_status,min_initial_quantity_status,max_initial_quantity_status,var_initial_quantity_status,median_initial_quantity_status,std_initial_quantity_status,q1_initial_quantity_status,q3_initial_quantity_status,mean_sold_quantity_status,min_sold_quantity_status,max_sold_quantity_status,var_sold_quantity_status,median_sold_quantity_status,std_sold_quantity_status,q1_sold_quantity_status,q3_sold_quantity_status,mean_base_price_listing_type_id,min_base_price_listing_type_id,max_base_price_listing_type_id,var_base_price_listing_type_id,median_base_price_listing_type_id,std_base_price_listing_type_id,q1_base_price_listing_type_id,q3_base_price_listing_type_id,mean_initial_quantity_listing_type_id,min_initial_quantity_listing_type_id,max_initial_quantity_listing_type_id,var_initial_quantity_listing_type_id,median_initial_quantity_listing_type_id,std_initial_quantity_listing_type_id,q1_initial_quantity_listing_type_id,q3_initial_quantity_listing_type_id,mean_sold_quantity_listing_type_id,min_sold_quantity_listing_type_id,max_sold_quantity_listing_type_id,var_sold_quantity_listing_type_id,median_sold_quantity_listing_type_id,std_sold_quantity_listing_type_id,q1_sold_quantity_listing_type_id,q3_sold_quantity_listing_type_id,mean_base_price_state_id,min_base_price_state_id,max_base_price_state_id,var_base_price_state_id,median_base_price_state_id,std_base_price_state_id,q1_base_price_state_id,q3_base_price_state_id,mean_initial_quantity_state_id,min_initial_quantity_state_id,max_initial_quantity_state_id,var_initial_quantity_state_id,median_initial_quantity_state_id,std_initial_quantity_state_id,q1_initial_quantity_state_id,q3_initial_quantity_state_id,mean_sold_quantity_state_id,min_sold_quantity_state_id,max_sold_quantity_state_id,var_sold_quantity_state_id,median_sold_quantity_state_id,std_sold_quantity_state_id,q1_sold_quantity_state_id,q3_sold_quantity_state_id,mean_base_price_automatic_relist,min_base_price_automatic_relist,max_base_price_automatic_relist,var_base_price_automatic_relist,median_base_price_automatic_relist,std_base_price_automatic_relist,q1_base_price_automatic_relist,q3_base_price_automatic_relist,mean_initial_quantity_automatic_relist,min_initial_quantity_automatic_relist,max_initial_quantity_automatic_relist,var_initial_quantity_automatic_relist,median_initial_quantity_automatic_relist,std_initial_quantity_automatic_relist,q1_initial_quantity_automatic_relist,q3_initial_quantity_automatic_relist,mean_sold_quantity_automatic_relist,min_sold_quantity_automatic_relist,max_sold_quantity_automatic_relist,var_sold_quantity_automatic_relist,median_sold_quantity_automatic_relist,std_sold_quantity_automatic_relist,q1_sold_quantity_automatic_relist,q3_sold_quantity_automatic_relist,mean_base_price_accepts_mercadopago,min_base_price_accepts_mercadopago,max_base_price_accepts_mercadopago,var_base_price_accepts_mercadopago,median_base_price_accepts_mercadopago,std_base_price_accepts_mercadopago,q1_base_price_accepts_mercadopago,q3_base_price_accepts_mercadopago,mean_initial_quantity_accepts_mercadopago,min_initial_quantity_accepts_mercadopago,ma

In [161]:
# Libreriras para modelado y AUTOML
# ==============================================================================
from pycaret.datasets import get_data
from pycaret.classification import *
from pycaret.classification import ClassificationExperiment
#from pycaret.utils import check_metric
from imblearn.over_sampling import *
import os

# Assuming you have already run predict_model and stored the result in `a`
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score, cohen_kappa_score, matthews_corrcoef


In [162]:
ML_new_or_used = load_model("data/Outputs/Modelos/ML_CLassification_condition_2024_06_16")
print(load_model("data/Outputs/Modelos/ML_CLassification_condition_2024_06_16"))

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['subtitle', 'month',
                                             'min_initial_quantity_mode',
                                             'max_initial_quantity_mode',
                                             'min_sold_quantity_mode',
                                             'median_sold_quantity_mode',
                                             'q1_sold_quantity_mode',
                                             'var_base_price_status',
                                             'min_initial_quantity_status',
                                             'max_initial_quantity_status',
                                             'median_initial_quan...
                 TransformerWrapper(transformer=RemoveOutliers(random_state=765))),
           

In [163]:
a = predict_model(ML_new_or_used, data=df_products_01, raw_score=True)

In [164]:
a.head(5)

,base_price,price,accepts_mercadopago,automatic_relist,initial_quantity,sold_quantity,available_quantity,local_pick_up,free_shipping,month,weekday,mean_base_price_mode,min_base_price_mode,max_base_price_mode,var_base_price_mode,median_base_price_mode,std_base_price_mode,q1_base_price_mode,q3_base_price_mode,mean_initial_quantity_mode,min_initial_quantity_mode,max_initial_quantity_mode,var_initial_quantity_mode,median_initial_quantity_mode,std_initial_quantity_mode,q1_initial_quantity_mode,q3_initial_quantity_mode,mean_sold_quantity_mode,min_sold_quantity_mode,max_sold_quantity_mode,var_sold_quantity_mode,median_sold_quantity_mode,std_sold_quantity_mode,q1_sold_quantity_mode,q3_sold_quantity_mode,mean_base_price_status,min_base_price_status,max_base_price_status,var_base_price_status,median_base_price_status,std_base_price_status,q1_base_price_status,q3_base_price_status,mean_initial_quantity_status,min_initial_quantity_status,max_initial_quantity_status,var_initial_quantity_status,median_initial_quantity_status,std_initial_quantity_status,q1_initial_quantity_status,q3_initial_quantity_status,mean_sold_quantity_status,min_sold_quantity_status,max_sold_quantity_status,var_sold_quantity_status,median_sold_quantity_status,std_sold_quantity_status,q1_sold_quantity_status,q3_sold_quantity_status,mean_base_price_listing_type_id,min_base_price_listing_type_id,max_base_price_listing_type_id,var_base_price_listing_type_id,median_base_price_listing_type_id,std_base_price_listing_type_id,q1_base_price_listing_type_id,q3_base_price_listing_type_id,mean_initial_quantity_listing_type_id,min_initial_quantity_listing_type_id,max_initial_quantity_listing_type_id,var_initial_quantity_listing_type_id,median_initial_quantity_listing_type_id,std_initial_quantity_listing_type_id,q1_initial_quantity_listing_type_id,q3_initial_quantity_listing_type_id,mean_sold_quantity_listing_type_id,min_sold_quantity_listing_type_id,max_sold_quantity_listing_type_id,var_sold_quantity_listing_type_id,median_sold_quantity_listing_type_id,std_sold_quantity_listing_type_id,q1_sold_quantity_listing_type_id,q3_sold_quantity_listing_type_id,mean_base_price_state_id,min_base_price_state_id,max_base_price_state_id,var_base_price_state_id,median_base_price_state_id,std_base_price_state_id,q1_base_price_state_id,q3_base_price_state_id,mean_initial_quantity_state_id,min_initial_quantity_state_id,max_initial_quantity_state_id,var_initial_quantity_state_id,median_initial_quantity_state_id,std_initial_quantity_state_id,q1_initial_quantity_state_id,q3_initial_quantity_state_id,mean_sold_quantity_state_id,min_sold_quantity_state_id,max_sold_quantity_state_id,var_sold_quantity_state_id,median_sold_quantity_state_id,std_sold_quantity_state_id,q1_sold_quantity_state_id,q3_sold_quantity_state_id,mean_base_price_automatic_relist,min_base_price_automatic_relist,max_base_price_automatic_relist,var_base_price_automatic_relist,median_base_price_automatic_relist,std_base_price_automatic_relist,q1_base_price_automatic_relist,q3_base_price_automatic_relist,mean_initial_quantity_automatic_relist,min_initial_quantity_automatic_relist,max_initial_quantity_automatic_relist,var_initial_quantity_automatic_relist,median_initial_quantity_automatic_relist,std_initial_quantity_automatic_relist,q1_initial_quantity_automatic_relist,q3_initial_quantity_automatic_relist,mean_sold_quantity_automatic_relist,min_sold_quantity_automatic_relist,max_sold_quantity_automatic_relist,var_sold_quantity_automatic_relist,median_sold_quantity_automatic_relist,std_sold_quantity_automatic_relist,q1_sold_quantity_automatic_relist,q3_sold_quantity_automatic_relist,mean_base_price_accepts_mercadopago,min_base_price_accepts_mercadopago,max_base_price_accepts_mercadopago,var_base_price_accepts_mercadopago,median_base_price_accepts_mercadopago,std_base_price_accepts_mercadopago,q1_base_price_accepts_mercadopago,q3_base_price_accepts_mercadopago,mean_initial_quantity_accepts_mercadopago,min_initial_quantity_accepts_mercadopago,ma

In [165]:
a['condition'] = y_test
# Reemplazar 'new' por 1 y 'used' por 0 en la df_ajust
a['target'] = a['condition'].replace({'new': 1, 'used': 0})

In [166]:
# Compute evaluation metrics
accuracy = accuracy_score(a['target'], a['prediction_label'])
auc = roc_auc_score(a['target'], a['prediction_score_1'])
recall = recall_score(a['target'], a['prediction_label'], average='binary')
precision = precision_score(a['target'], a['prediction_label'], average='binary')
f1 = f1_score(a['target'], a['prediction_label'], average='binary')
kappa = cohen_kappa_score(a['target'], a['prediction_label'])
mcc = matthews_corrcoef(a['target'], a['prediction_label'])

# Print or store the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1: {f1:.4f}")
print(f"Kappa: {kappa:.4f}")
print(f"MCC: {mcc:.4f}")

Accuracy: 0.7903
AUC: 0.8786
Recall: 0.8045
Precision: 0.8055
F1: 0.8050
Kappa: 0.5781
MCC: 0.5781
